In [149]:
import pandas as pd
import pyreadstat 
import chardet

In [150]:
#Import the csv
population = pd.read_csv('./Datasets/Populationbyregion.csv').drop_duplicates()
#Get rid of some of the forests that sit in different regions by adding them together
totals=population.groupby(['county','indicator'])['Value'].sum().reset_index()
#Pivot to get a wide rather than a narrow dataset
population_data = totals.pivot(index=['county'],columns='indicator', values='Value')
#The household size data is currently one significant figure, recalculate
population_data['Average Household size'] = population_data['Population, Total']/population_data['Number of Households']

In [151]:
mtf_battery_df, mtf_battery_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/MTF_HH_Battery_Data_Final.dta')
mtf_weight_df, mtf_weight_meta = pyreadstat.read_dta('./Datasets/Kenya_MTF_data/weight.dta')

In [152]:
with_weights = pd.merge(mtf_battery_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()
battery_hhs = pd.merge(max_vals,sum_vals,on = ['PARENT_KEY']).rename(columns={'c_c_110':'total_battery_capacity', 'c_c_111':'total_battery_voltage','c_multiple':'num_batteries'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals=with_weights.groupby(['PARENT_KEY'])['pw_final', 'c_multiple'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1801959063.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals=with_weights.groupby(['PARENT_KEY'])['c_c_110', 'c_c_111'].sum().reset_index()


In [153]:
mtf_core_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Core_survey_Final_data_trimmed-2.dta', convert_categoricals=False).drop(columns=['enum_id', 'enum_foot','enum_ht','sup_id','tab_id','subdate','start_time'])
with_weights_survey = pd.merge(mtf_core_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
mean_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['b_b_9','c_c_3','c_c_21','c_c_25ai','c_c_25aii'].mean().reset_index().rename(columns={'b_b_9':'rooms_occupied','c_c_3':'distance_to_grid','c_c_21':'avg_electricity_consumption_kwh','c_c_25ai':'worst_hours_electricity','c_c_25aii':'typical_hours_electricity'})
min_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])[ 'b_b_16','b_b_21','c_c_2','c_c_38','c_c_75','c_c_104'].min().reset_index().rename(columns={'b_b_16':'bank_account_held','b_b_21':'mobile_money_acc_held','c_c_2':'grid_connection','c_c_38':'mini_grid_connection','c_c_75':'generator_for_electricity','c_c_104':'rechargeable_battery_electricity'})
total_survey = pd.merge(pd.merge(max_vals_survey, mean_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc']), min_vals_survey, on = ['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])

/Users/rebecca.duke/miniforge3/lib/python3.9/site-packages/pandas/io/stata.py:1514: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727438286.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_survey=with_weights_survey.groupby(['PARENT_KEY','HH_code','prov','cty','dist','div','loc', 'subloc','class', 'locality_ur', 'marg', 'grid_loc'])['pw_final', 'a_a_1','b_b_6a','b_b_7'].max().reset_index().rename(columns={'a_a_1':'household_size','b_b_6a':'num_hhd_in_dwelling','b_b_7':'own_dwelling'})
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/1727

In [154]:
mtf_solar_df = pd.read_stata('./Datasets/Kenya_MTF_data/MTF_HH_Solar_Roster.dta')
with_weights_solar = pd.merge(mtf_solar_df, mtf_weight_df[['PARENT_KEY','pw_final']], on = 'PARENT_KEY')
max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()
solar_hhs = pd.merge(max_vals_solar,sum_vals_solar,on = ['PARENT_KEY']).rename(columns={'c_133_batt':'solar_battery_capacity', 'c_131_pwr':'total_solar_rating','c_position':'num_solar'})

/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  max_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['pw_final', 'c_position'].max().reset_index()
/var/folders/wh/kn94yryj1b35n6692fmfv3dc0000gq/T/ipykernel_67333/2366092681.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sum_vals_solar=with_weights_solar.groupby(['PARENT_KEY'])['c_131_pwr','c_133_batt'].sum().reset_index()


In [155]:
total_survey.head()

,PARENT_KEY,HH_code,prov,cty,dist,div,loc,subloc,class,locality_ur,...,distance_to_grid,avg_electricity_consumption_kwh,worst_hours_electricity,typical_hours_electricity,bank_account_held,mobile_money_acc_held,grid_connection,mini_grid_connection,generator_for_electricity,rechargeable_battery_electricity
0,uuid:0006ae15-e9cf-419e-ac14-0c66a739366e,2210103012_12,North Eastern,21.0,Garissa,Central,Central,Waberi,1.0,1.0,...,888.0,NaN,NaN,NaN,1.0,2.0,2.0,2.0,2.0,2.0
1,uuid:001b24c5-30b9-41fe-ac90-9e9e3e476935,4061402011_10,Central,6.0,Thika West,Thika West,Makongeni,Kamenu,1.0,1.0,...,NaN,9.2,NaN,NaN,2.0,1.0,1.0,2.0,2.0,2.0
2,uuid:0031732a-2efc-43e7-ba34-9447ddc31b32,5330201011_05,Rift Valley,33.0,Trans Nzoia West,Kiminini,Kiminini,Kiminini,1.0,2.0,...,3.0,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0
3,uuid:0042ae86-e063-4b44-a859-9c253d82bd38,5431305011_08,Rift Valley,43.0,Bomet,Sigor,Sigor,Areiyet,1.0,2.0,...,4.0,NaN,NaN,NaN,2.0,1.0,2.0,2.0,2.0,2.0
4,uuid:006014f9-c69e-48b2-8198-60cbbc894b59,5301701021_10,Rift Valley,30.0,Turkana South,Katilu,Katilu,Lokapel,1.0,2.0,...,77.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0


In [169]:
raw_mtf_data =pd.merge(total_survey,pd.merge(battery_hhs,solar_hhs, how='outer',on='PARENT_KEY').drop(columns=(['pw_final_x','pw_final_y'])), on='PARENT_KEY', how='left')

In [184]:
raw_mtf_data['weightedhousehold_inhabitants'] = raw_mtf_data.pw_final*raw_mtf_data.household_size
raw_mtf_data['weighted_hh_per_dwelling'] = raw_mtf_data.pw_final*(raw_mtf_data.num_hhd_in_dwelling+1)
raw_mtf_data.loc[raw_mtf_data.own_dwelling==2,'own_dwelling'] = 0
raw_mtf_data['weighted_owned_dwelling'] = raw_mtf_data.pw_final*raw_mtf_data.own_dwelling

In [183]:
raw_mtf_data.rooms_occupied.value_counts()

3.0     1221
2.0     1188
1.0      851
4.0      577
5.0      238
6.0      127
8.0       47
7.0       44
13.0      28
12.0      15
9.0        8
0.0        8
10.0       6
15.0       2
19.0       2
11.0       1
14.0       1
23.0       1
17.0       1
25.0       1
66.0       1
20.0       1
16.0       1
Name: rooms_occupied, dtype: int64

In [176]:
raw_mtf_data.columns

Index(['PARENT_KEY', 'HH_code', 'prov', 'cty', 'dist', 'div', 'loc', 'subloc',
       'class', 'locality_ur', 'marg', 'grid_loc', 'pw_final',
       'household_size', 'num_hhd_in_dwelling', 'own_dwelling',
       'rooms_occupied', 'distance_to_grid', 'avg_electricity_consumption_kwh',
       'worst_hours_electricity', 'typical_hours_electricity',
       'bank_account_held', 'mobile_money_acc_held', 'grid_connection',
       'mini_grid_connection', 'generator_for_electricity',
       'rechargeable_battery_electricity', 'num_batteries',
       'total_battery_capacity', 'total_battery_voltage', 'num_solar',
       'total_solar_rating', 'solar_battery_capacity'],
      dtype='object')

In [179]:
raw_mtf_data.groupby('dist')['pw_final'].sum().reset_index().rename(columns={'pw_final':'num_households'})

,dist,num_households
0,Baringo,56326.980225
1,Bomet,91233.927734
2,Bungoma North,64392.570435
3,Bungoma South,250068.255859
4,Bungoma West,60442.570129
...,...,...
119,Wajir West,54397.684570
120,Wareng,9.066787
121,West Pokot,81176.212494
122,Westlands,76516.719238
